# Using SVM to predict stock market

In [1]:
# impporting libraries

In [2]:
import pandas as pd
from pandas_datareader import data, wb
import matplotlib.pyplot as plt
import numpy as np
import pandas_datareader as pdr

In [3]:
# importing data from yahoo

In [4]:

start_date = pd.to_datetime('2015-01-01')
stop_date = pd.to_datetime('2017-05-11')
sp = pdr.data.get_data_yahoo('GOOGL', start_date, stop_date)
sp

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2015-01-02,532.599976,535.799988,527.880005,529.549988,1324000,529.549988
2015-01-05,527.150024,527.989990,517.750000,519.460022,2059100,519.460022
2015-01-06,520.500000,521.210022,505.549988,506.640015,2722800,506.640015
2015-01-07,510.950012,511.489990,503.649994,505.149994,2345900,505.149994
2015-01-08,501.510010,507.500000,495.019989,506.910004,3652700,506.910004
2015-01-09,508.179993,508.600006,498.649994,500.720001,2100000,500.720001
2015-01-12,499.239990,500.279999,490.910004,497.059998,2856900,497.059998
2015-01-13,502.570007,508.600006,497.260010,501.799988,3047900,501.799988
2015-01-14,500.420013,508.260010,498.160004,505.929993,2566700,505.929993


In [5]:
# defining a function to calculate statistics

In [6]:
def get_stats(s, n=252):
    s = s.dropna()
    wins = len(s[s>0])
    total_return = s.sum()
    losses = len(s[s<0])
    evens = len(s[s==0])
    mean_w = round(s[s>0].mean(), 3)
    mean_l = round(s[s<0].mean(), 3)
    win_r = wins/losses
    mean_trd = round(s.mean(), 3)
    sd = round(np.std(s), 3)
    max_l = round(s.min(), 3)
    max_w = round(s.max(), 3)
    sharpe_r = round((s.mean()/np.std(s))*np.sqrt(n), 4)
    cnt = len(s)
    print'Trades:', cnt,'\n tot_return (in %)', total_return,'\n Wins:', wins,'\nLosses:', losses, '\nMean Win:', mean_w,'\nMean Loss:', mean_l,'\nMax Win:', max_w,'\nMax Loss:', max_l,'\nMean', mean_trd,'\nStd Dev:', sd,'\n *********','\nSharpe Ratio:', sharpe_r,'\n *********'   
        
        


In [7]:
# plotng the variation of share pricces

In [8]:
sp_c = sp['Close']
fig, ax = plt.subplots(figsize=(15,10))
sp_c.plot(color='k')
plt.title("Alphabet", fontsize=20)

In [9]:
# variation of daily change

In [10]:
sp['Daily Change'] = pd.Series(sp['Close'] - sp['Open'])
print 'mean is %s' %(sp['Daily Change'].mean())
print 'std deviation is %s' %(round(np.std(sp['Daily Change']), 3))

mean is -0.0834174343434
std deviation is 7.591


In [11]:
# Any novice can apply three strategies 
#1) daily return - it depends on closing value of shares between two working days
#2) Intra day - it depends on open and close value of shares from each day.
#3) Over-night - It depends on the close value of a share and its open value on the next day.

# sharper ratio is - Used to judge returns on risk adjusted basis. Higher the better.

In [12]:
# we will calculate maximum returns we can get if we invest on each day.We will calculate sharper ratio for the above three scenerios.

In [13]:
long_day_rtn = ((sp['Close'] - sp['Close'].shift(1))/sp['Close'].shift(1))*100
long_id_rtn = ((sp['Close'] - sp['Open'])/sp['Open'])*100
long_on_rtn = ((sp['Open'] - sp['Close'].shift(1))/sp['Close'].shift(1))*100

In [14]:
get_stats(long_day_rtn)

Trades: 593 
 tot_return (in %) 65.4497337135 
 Wins: 316 
Losses: 277 
Mean Win: 1.021 
Mean Loss: -0.928 
Max Win: 16.258 
Max Loss: -5.414 
Mean 0.11 
Std Dev: 1.48 
 ********* 
Sharpe Ratio: 1.184 
 *********


In [15]:
get_stats(long_id_rtn)

Trades: 594 
 tot_return (in %) -3.91826250104 
 Wins: 306 
Losses: 288 
Mean Win: 0.768 
Mean Loss: -0.829 
Max Win: 3.574 
Max Loss: -5.259 
Mean -0.007 
Std Dev: 1.083 
 ********* 
Sharpe Ratio: -0.0967 
 *********


In [16]:
get_stats(long_on_rtn)

Trades: 593 
 tot_return (in %) 69.2960930515 
 Wins: 332 
Losses: 253 
Mean Win: 0.583 
Mean Loss: -0.49 
Max Win: 12.998 
Max Loss: -6.743 
Mean 0.117 
Std Dev: 1.079 
 ********* 
Sharpe Ratio: 1.7194 
 *********


In [17]:
# in the above scenerios we get good sharper ratio for over night return model and daily return model.
# Our aim is to increase the sharper ratio using SVM.

In [18]:
# now to use SVM for prediction we will use last 20 values.
# For this purpose will make following matrix

In [19]:
for i in range(1, 21, 1):
    sp.loc[:,'Close Minus ' + str(i)] = sp['Close'].shift(i)
    sp20 = sp[[x for x in sp.columns if 'Close Minus' in x or x =='Close']].iloc[20:,]
sp20

,Close,Close Minus 1,Close Minus 2,Close Minus 3,Close Minus 4,Close Minus 5,Close Minus 6,Close Minus 7,Close Minus 8,Close Minus 9,...,Close Minus 11,Close Minus 12,Close Minus 13,Close Minus 14,Close Minus 15,Close Minus 16,Close Minus 17,Close Minus 18,Close Minus 19,Close Minus 20
Date,,,,,,,,,,,,,,,,,,,,,
2015-02-02,532.200012,537.549988,513.229980,512.429993,521.190002,536.719971,541.950012,537.299988,520.390015,509.940002,...,504.010010,505.929993,501.799988,497.059998,500.720001,506.910004,505.149994,506.640015,519.460022,529.549988
2015-02-03,533.299988,532.200012,537.549988,513.229980,512.429993,521.190002,536.719971,541.950012,537.299988,520.390015,...,510.459991,504.010010,505.929993,501.799988,497.059998,500.720001,506.910004,505.149994,506.640015,519.460022
2015-02-04,526.099976,533.299988,532.200012,537.549988,513.229980,512.429993,521.190002,536.719971,541.950012,537.299988,...,509.940002,510.459991,504.010010,505.929993,501.799988,497.059998,500.720001,506.910004,505.149994,506.640015
2015-02-05,529.830017,526.099976,533.299988,532.200012,537.549988,513.229980,512.429993,521.190002,536.719971,541.950012,...,520.390015,509.940002,510.459991,504.010010,505.929993,501.799988,497.059998,500.720001,506.910004,505.149994
2015-02-06,533.880005,529.830017,526.099976,533.299988,532.200012,537.549988,513.229980,512.429993,521.190002,536.719971,...,537.299988,520.390015,509.940002,510.459991,504.010010,505.929993,501.799988,497.059998,500.720001,506.910004
2015-02-09,529.280029,533.880005,529.830017,526.099976,533.299988,532.200012,537.549988,513.229980,512.429993,521.190002,...,541.950012,537.299988,520.390015,509.940002,510.459991,504.010010,505.929993,501.799988,497.059998,500.720001
2015-02-10,540.159973,529.280029,533.880005,529.830017,526.099976,533.299988,532.200012,537.549988,513.229980,512.429993,...,536.719971,541.950012,537.299988,520.390015,509.940002,510.459991,504.010010,505.929993,501.799988,497.059998
2015-02-11,538.000000,540.159973,529.280029,533.880005,529.830017,526.099976,533.299988,532.200012,537.549988,513.229980,...,521.190002,536.719971,541.950012,537.299988,520.390015,509.940002,510.459991,504.010010,505.929993,501.799988
2015-02-12,546.010010,538.000000,540.159973,529.280029,533.880005,529.830017,526.099976,533.299988,532.200012,537.549988,...,512.429993,521.190002,536.719971,541.950012,537.299988,520.390015,509.940002,510.459991,504.010010,505.929993


In [20]:
# changing the direction of row so that oldest value will be the first column.

In [21]:
sp20 = sp20.iloc[:,::-1]
sp20

,Close Minus 20,Close Minus 19,Close Minus 18,Close Minus 17,Close Minus 16,Close Minus 15,Close Minus 14,Close Minus 13,Close Minus 12,Close Minus 11,...,Close Minus 9,Close Minus 8,Close Minus 7,Close Minus 6,Close Minus 5,Close Minus 4,Close Minus 3,Close Minus 2,Close Minus 1,Close
Date,,,,,,,,,,,,,,,,,,,,,
2015-02-02,529.549988,519.460022,506.640015,505.149994,506.910004,500.720001,497.059998,501.799988,505.929993,504.010010,...,509.940002,520.390015,537.299988,541.950012,536.719971,521.190002,512.429993,513.229980,537.549988,532.200012
2015-02-03,519.460022,506.640015,505.149994,506.910004,500.720001,497.059998,501.799988,505.929993,504.010010,510.459991,...,520.390015,537.299988,541.950012,536.719971,521.190002,512.429993,513.229980,537.549988,532.200012,533.299988
2015-02-04,506.640015,505.149994,506.910004,500.720001,497.059998,501.799988,505.929993,504.010010,510.459991,509.940002,...,537.299988,541.950012,536.719971,521.190002,512.429993,513.229980,537.549988,532.200012,533.299988,526.099976
2015-02-05,505.149994,506.910004,500.720001,497.059998,501.799988,505.929993,504.010010,510.459991,509.940002,520.390015,...,541.950012,536.719971,521.190002,512.429993,513.229980,537.549988,532.200012,533.299988,526.099976,529.830017
2015-02-06,506.910004,500.720001,497.059998,501.799988,505.929993,504.010010,510.459991,509.940002,520.390015,537.299988,...,536.719971,521.190002,512.429993,513.229980,537.549988,532.200012,533.299988,526.099976,529.830017,533.880005
2015-02-09,500.720001,497.059998,501.799988,505.929993,504.010010,510.459991,509.940002,520.390015,537.299988,541.950012,...,521.190002,512.429993,513.229980,537.549988,532.200012,533.299988,526.099976,529.830017,533.880005,529.280029
2015-02-10,497.059998,501.799988,505.929993,504.010010,510.459991,509.940002,520.390015,537.299988,541.950012,536.719971,...,512.429993,513.229980,537.549988,532.200012,533.299988,526.099976,529.830017,533.880005,529.280029,540.159973
2015-02-11,501.799988,505.929993,504.010010,510.459991,509.940002,520.390015,537.299988,541.950012,536.719971,521.190002,...,513.229980,537.549988,532.200012,533.299988,526.099976,529.830017,533.880005,529.280029,540.159973,538.000000
2015-02-12,505.929993,504.010010,510.459991,509.940002,520.390015,537.299988,541.950012,536.719971,521.190002,512.429993,...,537.549988,532.200012,533.299988,526.099976,529.830017,533.880005,529.280029,540.159973,538.000000,546.010010


In [22]:
# Dividing data in test and training set

In [23]:
from sklearn.svm import SVR
clf = SVR(kernel='linear')
X_train = sp20[:-100]
y_train = sp20['Close'].shift(-1)[:-100]
X_test = sp20[-100:]
y_test = sp20['Close'].shift(-1)[-100:]

In [24]:
# Predicting the based on trained model

In [25]:
model = clf.fit(X_train, y_train)
preds = model.predict(X_test)

In [26]:
# creating a dataframe of the predicted and observed values

In [27]:
tf = pd.DataFrame(list(zip(y_test, preds)), columns=['Next Day Close','Predicted Next Close'], index=y_test.index)
tf['difference_close'] = tf['Predicted Next Close'] - tf['Next Day Close']
tf

,Next Day Close,Predicted Next Close,difference_close
Date,,,
2016-12-16,812.500000,810.180836,-2.319164
2016-12-19,815.200012,812.954305,-2.245707
2016-12-20,812.200012,817.799112,5.599100
2016-12-21,809.679993,812.028193,2.348200
2016-12-22,807.799988,807.335326,-0.464662
2016-12-23,809.929993,807.121397,-2.808596
2016-12-27,804.570007,810.854502,6.284495
2016-12-28,802.880005,802.151771,-0.728234
2016-12-29,792.450012,801.528153,9.078141


In [28]:
# modifing the data frame

In [29]:

ndo = sp[['Open']].iloc[-100:].shift(-1)

tf2 = pd.merge(tf, ndo, left_index=True, right_index=True)
tf2.columns = ['Next Day Close', 'Predicted Next Close','difference_close', 'Next Day Open']
tf2

,Next Day Close,Predicted Next Close,difference_close,Next Day Open
Date,,,,
2016-12-16,812.500000,810.180836,-2.319164,809.280029
2016-12-19,815.200012,812.954305,-2.245707,813.369995
2016-12-20,812.200012,817.799112,5.599100,815.719971
2016-12-21,809.679993,812.028193,2.348200,809.099976
2016-12-22,807.799988,807.335326,-0.464662,808.010010
2016-12-23,809.929993,807.121397,-2.808596,808.679993
2016-12-27,804.570007,810.854502,6.284495,813.330017
2016-12-28,802.880005,802.151771,-0.728234,802.330017
2016-12-29,792.450012,801.528153,9.078141,803.210022


In [30]:
# calculating factor a1 which will help us in setting criteria

In [31]:
a1 = tf2['Predicted Next Close'] - tf2['Next Day Open']
a1 = abs(a1[a1 <0].mean())
a1

3.580493557713129

In [32]:
# decision criteria

In [33]:
def get_signal(r):
    if r['Predicted Next Close'] > r['Next Day Open'] + a1:
        return 1
    else:
        return 0

In [34]:
def get_ret(r):
    if r['Signal'] == 1:
        return ((r['Next Day Close'] - r['Next Day Open'])/r['Next Day Open']) * 100
    else:
        return 0

In [35]:
tf2 = tf2.assign(Signal = tf2.apply(get_signal, axis=1))
tf2 = tf2.assign(PnL = tf2.apply(get_ret, axis=1))
tf2

,Next Day Close,Predicted Next Close,difference_close,Next Day Open,Signal,PnL
Date,,,,,,
2016-12-16,812.500000,810.180836,-2.319164,809.280029,0,0.000000
2016-12-19,815.200012,812.954305,-2.245707,813.369995,0,0.000000
2016-12-20,812.200012,817.799112,5.599100,815.719971,0,0.000000
2016-12-21,809.679993,812.028193,2.348200,809.099976,0,0.000000
2016-12-22,807.799988,807.335326,-0.464662,808.010010,0,0.000000
2016-12-23,809.929993,807.121397,-2.808596,808.679993,0,0.000000
2016-12-27,804.570007,810.854502,6.284495,813.330017,0,0.000000
2016-12-28,802.880005,802.151771,-0.728234,802.330017,0,0.000000
2016-12-29,792.450012,801.528153,9.078141,803.210022,0,0.000000


In [36]:
# output of svmm model prediction

In [37]:
get_stats(tf2['PnL'])

Trades: 100 
 tot_return (in %) 1.67462950613 
 Wins: 3 
Losses: 1 
Mean Win: 0.627 
Mean Loss: -0.207 
Max Win: 1.258 
Max Loss: -0.207 
Mean 0.017 
Std Dev: 0.138 
 ********* 
Sharpe Ratio: 1.9308 
 *********


In [38]:
#out put of previous three strategies

In [39]:
get_stats(long_day_rtn)
get_stats(long_id_rtn)
get_stats(long_on_rtn)

Trades: 593 
 tot_return (in %) 65.4497337135 
 Wins: 316 
Losses: 277 
Mean Win: 1.021 
Mean Loss: -0.928 
Max Win: 16.258 
Max Loss: -5.414 
Mean 0.11 
Std Dev: 1.48 
 ********* 
Sharpe Ratio: 1.184 
 *********
Trades: 594 
 tot_return (in %) -3.91826250104 
 Wins: 306 
Losses: 288 
Mean Win: 0.768 
Mean Loss: -0.829 
Max Win: 3.574 
Max Loss: -5.259 
Mean -0.007 
Std Dev: 1.083 
 ********* 
Sharpe Ratio: -0.0967 
 *********
Trades: 593 
 tot_return (in %) 69.2960930515 
 Wins: 332 
Losses: 253 
Mean Win: 0.583 
Mean Loss: -0.49 
Max Win: 12.998 
Max Loss: -6.743 
Mean 0.117 
Std Dev: 1.079 
 ********* 
Sharpe Ratio: 1.7194 
 *********


In [40]:
# for the given scenerios SVM model is least risky as it is the one with highest sharper ratio.

# END